### `functools` — Higher-order functions and operations on callable objects

#### 1. reduce()

> **Signature**: `functools.reduce(function, iterable[, initializer])`

Apply function of two arguments cumulatively to the items of iterable, from left to right, so as to reduce the iterable to a single value. For example, `reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])` calculates `((((1+2)+3)+4)+5)`. The `left` argument, `x`, is the `accumulated` value and the `right` argument, y, is the update value from the iterable. If the optional initializer is present, it is placed before the items of the iterable in the calculation, and serves as a default when the iterable is empty. If initializer is not given and iterable contains only one item, the first item is returned.

Roughly equivalent to:

```py
# function is assume operators.add, yeilding result per iteration 
def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value
```

In [1]:
from functools import reduce

In [2]:
reduce(lambda x, y: x + y, [1,2,3,4,5]) # reduce(func, iterable, *initializer) # initializer is optional

15

In [3]:
reduce(lambda x, y: x + y, [1,2,3,4,5], 100) # pass with initializer

115

In [4]:
# or
from operator import add
reduce(add, [10,20,30,40], 100) # pass operators.add function as func parameter

200

In [5]:
# or
sum([10,20,30,40], 100)

200

In [6]:
opt_list = [max, min, add]

for opt in opt_list:
    print(reduce(lambda x, y: opt(x, y), [1,2,3,4,5]))

5
1
15


---
#### 2. @functools.total_ordering
Given a class defining one or more rich comparison ordering methods, this class decorator supplies the rest. This simplifies the effort involved in specifying all of the possible rich comparison operations:

The class must define one of `__lt__()`, `__le__()`, `__gt__()`, or `__ge__()`. In addition, the class `should supply` an `__eq__()` method.

- `__eq__` : `==`
- `__le__` : `<=`
- `__gt__` : `>`
- `__ge__` : `>=`
- `__lt__` : `<`

and more...

In [7]:
from functools import total_ordering

In [8]:
@total_ordering
class Car:
    def __init__(self, model, mileage):
        self.model = model
        self.mileage = mileage
    
    # required one      
    def __lt__(self, other):
        return self.mileage < other.mileage
    
    # required one     
    def __eq__(self, other):
        return self.mileage == other.mileage
    
    # ... we don't implement other odering magic/dunder method
    # ... istead of implement we use @total_ordering decorator     

In [9]:
c1 = Car('BMW', 90)
c2 = Car('Ford', 70)

In [10]:
c1 == c2

False

In [11]:
c1 > c2, c1 < c2, c1 >= c2, c1 <= c2, c1 != c2 # we don't implement all of that but all works

(True, False, True, False, True)

---
#### 3. @functools.cached_property(func)
Transform a method of a class into a property whose value is computed once and then cached as a normal attribute for the life of the instance. Similar to `property()`, with the addition of caching. Useful for expensive computed properties of instances that are otherwise effectively immutable.

Example:

```py
class DataSet:

    def __init__(self, sequence_of_numbers):
        self._data = tuple(sequence_of_numbers)

    @cached_property
    def stdev(self):
        return statistics.stdev(self._data)
```

#### Same as above

If a mutable mapping is not available or if space-efficient key sharing is desired, an effect similar to cached_property() can be achieved by a stacking property() on top of cache():


```py
class DataSet:
    def __init__(self, sequence_of_numbers):
        self._data = sequence_of_numbers

    @property
    @cache
    def stdev(self):
        return statistics.stdev(self._data)
```

##### First understand @property

[@property](https://docs.python.org/3/library/functions.html#property)

In [12]:
class Marksheet:
    def __init__(self, *args):
        self.marks = args
    
    @property
    def total(self):
        print("Calculating total: ")
        print(sum(self.marks))
    
    @property
    def average(self):
        print("Calculating average: ")
        print(sum(self.marks) / len(self.marks))
    
    def min(self):
        return min(self.marks)

In [13]:
m = Marksheet(90,80, 89)

In [14]:
m.total # we don't need to call it like `m.total()`

Calculating total: 
259


In [15]:
m.average # don't need to use first () brackets

Calculating average: 
86.33333333333333


In [16]:
m.min() # using  first () brackets. Because this is a class method.

80

##### Now understand @cache_property

In [17]:
from functools import cached_property

In [18]:
class Marksheet:
    def __init__(self, *marks):
        self.marks = marks
    
    @cached_property
    def total(self):
        print("Calculating total: ")
        return sum(self.marks)
    
    @property
    def average(self):
        print("Calculating average: ")
        return sum(self.marks) / len(self.marks)

In [19]:
mark = Marksheet(59, 89, 99)

In [20]:
mark.total # call class property

Calculating total: 


247

In [21]:
mark.total # didn't call the property, return the cache_result

247

In [22]:
mark.average

Calculating average: 


82.33333333333333

In [23]:
mark.average # I hope now you understand clearly the @cache_property

Calculating average: 


82.33333333333333

---
#### 4. @functools.lru_cache(maxsize=128, typed=False)

@functools.lru_cache(user_function)

Decorator to wrap a function with a `memoizing callable` that saves up to the maxsize most recent calls. It can save time when an expensive or I/O bound function is periodically called with the same arguments.

Since a dictionary is used to cache results, the positional and keyword arguments to the function must be hashable.

Distinct argument patterns may be considered to be distinct calls with `separate` cache entries. For example, `f(a=1, b=2)` and `f(b=2, a=1)` differ in their keyword argument order and may have two separate cache entries.

If user_function is specified, it must be a callable. This allows the lru_cache decorator to be applied directly to a user function, leaving the maxsize at its default value of 128:

```py
@lru_cache
def count_vowels(sentence):
    sentence = sentence.casefold()
    return sum(sentence.count(vowel) for vowel in 'aeiou')
```

If maxsize is set to `None`, the `LRU feature is disabled and the cache can grow without bound`.

If `typed` is set to `true`, function arguments of different types will be cached `separately`. For example, `f(3)` and `f(3.0)` will be treated as distinct calls with distinct results.

In [24]:
from functools import lru_cache
@lru_cache(maxsize=3)
def fib(n):
    if n < 2:
        return n
    print(f'Calculating fib({n})')
    return fib(n-1) + fib(n-2)

In [25]:
[fib(x) for x in range(15)]

Calculating fib(2)
Calculating fib(3)
Calculating fib(4)
Calculating fib(5)
Calculating fib(6)
Calculating fib(7)
Calculating fib(8)
Calculating fib(9)
Calculating fib(10)
Calculating fib(11)
Calculating fib(12)
Calculating fib(13)
Calculating fib(14)


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]

In [26]:
fib.cache_info()

CacheInfo(hits=26, misses=15, maxsize=3, currsize=3)